### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.
  

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Dense
import pickle

In [4]:
data = pd.read_csv('Churn_Modelling.csv')

In [5]:
data = data.drop(['Surname', 'RowNumber', 'CustomerId'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [8]:
lb = LabelEncoder()
data['Gender'] = lb.fit_transform(data['Gender'])

In [9]:
OHE = OneHotEncoder()
geo_encoder = OHE.fit_transform(data[['Geography']])

In [10]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(), columns=OHE.get_feature_names_out(['Geography']))

In [11]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoder_df], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [12]:
X= data.drop(['Exited'], axis=1)
Y= data['Exited']

In [13]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [14]:
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

In [15]:
with open('label_encoder.pkl', 'wb')as file:
    pickle.dump(lb, file)
with open('OneHotEncoder.pkl', 'wb') as file:
    pickle.dump(OHE, file)
with open('scaler.pkl', 'wb' ) as file:
    pickle.dump(scale, file)


In [26]:
def create_model(neurons =32, layers =1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape =(X_train.shape[1],) ))

    for _ in range(layers-1):
        model.add(Dense(neurons, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ['accuracy'])

    return model

In [31]:
model = KerasClassifier(neurons=32, layers=1, build_fn=create_model, epochs = 50, batch_size=10, verbose=2)
## Verbose is bascially amount of output shown while training the model.

In [38]:
param = {
    'neurons' : [16,32,64],
    'layers' : [1,2],
    'epochs' : [10,20]
}

In [39]:
grid = GridSearchCV(estimator=model, param_grid=param, n_jobs=-1, cv=3) 
### n_jobs = 1 uses only 1 core on processor if n_jobs=-1 use whatever avaiable cores (parralelism)

##cv how many folds to split data for cross-validation inside grid search.
## 3 me se 2 for trainig and 1 for testing

In [40]:
grid_result = grid.fit(X_train, Y_train)

print(grid_result.best_score_, grid_result.best_params_)

/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. 

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10


/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
534/534 - 2s - 4ms/step - accuracy: 0.8031 - loss: 0.4663
Epoch 2/10
534/534 - 2s - 4ms/step - accuracy: 0.7915 - loss: 0.4745
Epoch 2/10
534/534 - 2s - 4ms/step - accuracy: 0.7868 - loss: 0.4840
Epoch 2/10
534/534 - 2s - 4ms/step - accuracy: 0.7703 - loss: 0.5047
Epoch 2/10
534/534 - 2s - 4ms/step - accuracy: 0.7066 - loss: 0.5706
Epoch 2/10
534/534 - 2s - 4ms/step - accuracy: 0.7889 - loss: 0.4726
Epoch 2/10
534/534 - 2s - 5ms/step - accuracy: 0.8024 - loss: 0.4553
Epoch 2/10
534/534 - 3s - 5ms/step - accuracy: 0.7219 - loss: 0.5542
Epoch 2/10
534/534 - 1s - 2ms/step - accuracy: 0.8296 - loss: 0.4049
Epoch 3/10
534/534 - 1s - 2ms/step - accuracy: 0.8228 - loss: 0.4193
Epoch 3/10
534/534 - 1s - 2ms/step - accuracy: 0.8279 - loss: 0.4022
Epoch 3/10
534/534 - 1s - 2ms/step - accuracy: 0.8163 - loss: 0.4260
Epoch 3/10
534/534 - 1s - 2ms/step - accuracy: 0.8161 - loss: 0.4263
Epoch 3/10
534/534 - 1s - 2ms/step - accuracy: 0.8243 - loss: 0.4105
Epoch 3/10
534/534 - 1s - 2ms/step

/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/keras/src/layers/

...

534/534 - 1s - 2ms/step - accuracy: 0.8669 - loss: 0.3192
Epoch 18/20
Epoch 1/20
534/534 - 1s - 2ms/step - accuracy: 0.8611 - loss: 0.3306
Epoch 7/20
534/534 - 1s - 2ms/step - accuracy: 0.8614 - loss: 0.3305
534/534 - 1s - 2ms/step - accuracy: 0.8650 - loss: 0.3212
534/534 - 1s - 2ms/step - accuracy: 0.8630 - loss: 0.3259
Epoch 9/20
534/534 - 1s - 2ms/step - accuracy: 0.8672 - loss: 0.3079
Epoch 20/20
534/534 - 1s - 2ms/step - accuracy: 0.8682 - loss: 0.3246
Epoch 19/20
534/534 - 1s - 2ms/step - accuracy: 0.8672 - loss: 0.3171
Epoch 19/20
267/267 - 0s - 2ms/step
534/534 - 1s - 2ms/step - accuracy: 0.8596 - loss: 0.3256
Epoch 8/20


/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


267/267 - 1s - 2ms/step
Epoch 1/20
534/534 - 1s - 2ms/step - accuracy: 0.8650 - loss: 0.3235
Epoch 10/20
534/534 - 1s - 2ms/step - accuracy: 0.8738 - loss: 0.3059
534/534 - 1s - 2ms/step - accuracy: 0.8691 - loss: 0.3239
Epoch 20/20
534/534 - 1s - 2ms/step - accuracy: 0.8687 - loss: 0.3157
Epoch 20/20
534/534 - 1s - 2ms/step - accuracy: 0.8650 - loss: 0.3228
Epoch 9/20
267/267 - 0s - 2ms/step
534/534 - 1s - 2ms/step - accuracy: 0.8675 - loss: 0.3200
Epoch 11/20
534/534 - 3s - 5ms/step - accuracy: 0.8146 - loss: 0.4344
Epoch 2/20
534/534 - 1s - 2ms/step - accuracy: 0.8654 - loss: 0.3240
534/534 - 1s - 2ms/step - accuracy: 0.8727 - loss: 0.3143
534/534 - 1s - 2ms/step - accuracy: 0.8609 - loss: 0.3195
Epoch 10/20
267/267 - 0s - 1ms/step
534/534 - 1s - 2ms/step - accuracy: 0.8680 - loss: 0.3182
Epoch 12/20
534/534 - 2s - 4ms/step - accuracy: 0.8168 - loss: 0.4269
Epoch 2/20
267/267 - 0s - 1ms/step
534/534 - 1s - 2ms/step - accuracy: 0.8543 - loss: 0.3632
Epoch 3/20
534/534 - 1s - 2ms/step

/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/yashjain/Downloads/Generative-AI/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


800/800 - 1s - 1ms/step - accuracy: 0.7958 - loss: 0.4592
Epoch 2/20
800/800 - 0s - 557us/step - accuracy: 0.8449 - loss: 0.3829
Epoch 3/20
800/800 - 0s - 474us/step - accuracy: 0.8519 - loss: 0.3592
Epoch 4/20
800/800 - 0s - 471us/step - accuracy: 0.8568 - loss: 0.3504
Epoch 5/20
800/800 - 0s - 463us/step - accuracy: 0.8593 - loss: 0.3457
Epoch 6/20
800/800 - 0s - 467us/step - accuracy: 0.8596 - loss: 0.3432
Epoch 7/20
800/800 - 0s - 457us/step - accuracy: 0.8619 - loss: 0.3415
Epoch 8/20
800/800 - 0s - 461us/step - accuracy: 0.8624 - loss: 0.3395
Epoch 9/20
800/800 - 0s - 458us/step - accuracy: 0.8614 - loss: 0.3383
Epoch 10/20
800/800 - 0s - 458us/step - accuracy: 0.8630 - loss: 0.3375
Epoch 11/20
800/800 - 0s - 459us/step - accuracy: 0.8626 - loss: 0.3364
Epoch 12/20
800/800 - 0s - 458us/step - accuracy: 0.8626 - loss: 0.3346
Epoch 13/20
800/800 - 0s - 461us/step - accuracy: 0.8633 - loss: 0.3347
Epoch 14/20
800/800 - 0s - 457us/step - accuracy: 0.8624 - loss: 0.3336
Epoch 15/20
80